In [ ]:
# V2V Scenario 36 - 64 Beams

In [ ]:
import os
import random
import datetime
import sys
import shutil

import torch
import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transf
from torchvision import transforms, utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import ast 

In [ ]:
########################################################################
# Create save directory
########################################################################

In [ ]:
# year month day 
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds 
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

In [ ]:
pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime 
print(pwd)
# Determine whether the folder already exists
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

In [ ]:
# Copy the training files to the saved directory
shutil.copy('./scenario36_64_pos_beam_train.csv', pwd)
shutil.copy('./scenario36_64_pos_beam_val.csv', pwd)
shutil.copy('./scenario36_64_pos_beam_test.csv', pwd)

In [ ]:
# Create folder to save analysis files and checkpoint

In [ ]:
save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory) 
        
isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

In [ ]:
########################################################################
# Data Feeding: Create data sample list
########################################################################

In [ ]:
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        data = list(row.values[1:])
        data_samples.append(data)

    return data_samples

In [ ]:
class DataFeed(Dataset):
    '''
    A class retrieving a tuple of (image,label). It can handle the case
    of empty classes (empty folders).
    '''
    def __init__(self,root_dir, nat_sort = False, transform=None, init_shuflle = True):
        self.root = root_dir
        self.samples = create_samples(self.root,shuffle=init_shuflle,nat_sort=nat_sort)
        self.transform = transform


    def __len__(self):
        return len( self.samples )

    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        pos_val = sample[:1]
        pos_val = ast.literal_eval(pos_val[0])    
        pos_val = np.asarray(pos_val)

        pos_centers = sample[1:2]
        pos_centers = np.asarray(pos_centers)

        return (pos_val, pos_centers)

In [ ]:
########################################################################
# Training, Testing, and Validation!
########################################################################

In [ ]:
# Hyperparameters

In [ ]:
# Training Hyper-parameters
batch_size = 128
val_batch_size = 1
lr = 0.01
decay = 1e-4
num_epochs = 30
train_size = [1]

# Hyperparameters for our network
input_size = 2
node = 512
output_size = 65

In [ ]:
# Training and Validation

In [ ]:
proc_pipe = transf.Compose(
    [
        transf.ToTensor()
    ]
)

train_dir = './scenario36_64_pos_beam_train.csv'
val_dir = './scenario36_64_pos_beam_val.csv'

train_loader = DataLoader(DataFeed(train_dir, transform = proc_pipe),
                            batch_size=batch_size,
                              #num_workers=8,
                            shuffle=False)
val_loader = DataLoader(DataFeed(val_dir, transform=proc_pipe),
                        batch_size=val_batch_size,
                        #num_workers=8,
                        shuffle=False)

In [ ]:
########################################################################
# Main Model
########################################################################

In [ ]:
class NN_beam_pred(nn.Module):
    def __init__(self, num_features, num_output):
        super(NN_beam_pred, self).__init__()
            
        self.cnn_layers = nn.Sequential(
            # Defining the convolution layer
            nn.Conv1d(num_features, 30, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(30, 20, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(20, 10, kernel_size=2, stride=1, padding=1),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(10, 30, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(30, 20, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(20, 10, kernel_size=2, stride=1, padding=1),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(10, 30, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(30, 20, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(20, 10, kernel_size=2, stride=1, padding=1),
            nn.MaxPool1d(kernel_size=2, stride=2),
      
        )
        
        self.l1 = nn.Linear(20, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, num_output)
            
    def forward(self, inputs):
        x = self.cnn_layers(inputs)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
model = NN_beam_pred(input_size, num_output=output_size)  

In [ ]:
val_acc = []

with cuda.device(0):
    top_1 = np.zeros( (1,len(train_size)) )
    top_3 = np.zeros( (1,len(train_size)) )
    top_5 = np.zeros( (1,len(train_size)) )
    top_7 = np.zeros( (1,len(train_size)) )
    top_9 = np.zeros( (1,len(train_size)) )
    top_11 = np.zeros( (1,len(train_size)) )
    top_13 = np.zeros( (1,len(train_size)) )
    top_15 = np.zeros( (1,len(train_size)) )
    acc_loss = 0
    itr = []
    for idx, n in enumerate(train_size):
        print('```````````````````````````````````````````````````````')
        print('Training size is {}'.format(n))
        # Build the network:
        net = model.cuda()
        layers = list(net.children())
    
        #  Optimization parameters:
        criterion = nn.CrossEntropyLoss()
        opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
        LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [15,25,40], gamma=0.1, last_epoch=-1)
    
        count = 0
        running_loss = []
        running_top1_acc = []  
        running_top3_acc = []
        running_top5_acc = []
        running_top7_acc = []
        running_top9_acc = []
        running_top11_acc = []
        running_top13_acc = []
        running_top15_acc = []
        
        best_accuracy = 0
            
        for epoch in range(num_epochs):
            print('Epoch No. ' + str(epoch + 1))
            skipped_batches = 0
            for tr_count, (pos_data, beam_val) in enumerate(train_loader):
                net.train()
                data = pos_data.type(torch.Tensor)                  
                label = beam_val[:,0].type(torch.LongTensor)                    
                x = data.cuda()
                # print("x size", x.size())
                opt.zero_grad()
                label = label.cuda()
                # print("label size", label.size())
                # print(x.shape)
                out = net.forward(x.reshape([-1, 2, 1]))
                # print(out.shape, label.shape)
                loss = criterion(out, label)
                loss.backward()
                opt.step()                    
                batch_loss = loss.item()
                acc_loss += batch_loss
                count += 1
                if np.mod(count, 100) == 0:
                    print('Training-Batch No.' + str(count))
                    running_loss.append(batch_loss)  # running_loss.append()
                    itr.append(count)
                    print('Loss = ' + str(running_loss[-1]))
    
            print('Start validation')
            ave_top1_acc = 0
            ave_top3_acc = 0
            ave_top5_acc = 0
            ave_top7_acc = 0
            ave_top9_acc = 0
            ave_top11_acc = 0
            ave_top13_acc = 0
            ave_top15_acc = 0
            ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
            top1_pred_out = []
            top3_pred_out = []
            top5_pred_out = []
            top7_pred_out = []
            top9_pred_out = []
            top11_pred_out = []
            top13_pred_out = []
            top15_pred_out = []
            total_count = 0

            gt_beam = []
            for val_count, (pos_data, beam_val) in enumerate(val_loader):
                net.eval()
                data = pos_data.type(torch.Tensor)  
                x = data.cuda()                    
                labels = beam_val[:,0].type(torch.LongTensor)   
                opt.zero_grad()
                labels = labels.cuda()
                gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
                total_count += labels.size(0)
                out = net.forward(x.reshape([-1, 2, 1]))
                _, top_1_pred = t.max(out, dim=1)
                top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
                sorted_out = t.argsort(out, dim=1, descending=True)
 
                top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())
        
                top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())
                
                top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())
                
                top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())
                
                top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())
                
                top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())
                
                top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())
                        
                reshaped_labels = labels.reshape((labels.shape[0], 1))
                tiled_3_labels = reshaped_labels.repeat(1, 3)
                tiled_5_labels = reshaped_labels.repeat(1, 5)
                tiled_7_labels = reshaped_labels.repeat(1, 7)
                tiled_9_labels = reshaped_labels.repeat(1, 9)
                tiled_11_labels = reshaped_labels.repeat(1, 11)
                tiled_13_labels = reshaped_labels.repeat(1, 13)
                tiled_15_labels = reshaped_labels.repeat(1, 15)
                   
                batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
                batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
                batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
                batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
                batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
                batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
                batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
                batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

                ave_top1_acc += batch_top1_acc.item()
                ave_top3_acc += batch_top3_acc.item()
                ave_top5_acc += batch_top5_acc.item()
                ave_top7_acc += batch_top7_acc.item()
                ave_top9_acc += batch_top9_acc.item()
                ave_top11_acc += batch_top11_acc.item()
                ave_top13_acc += batch_top13_acc.item()
                ave_top15_acc += batch_top15_acc.item()
            
            print("total test examples are", total_count)
            running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
            running_top3_acc.append(ave_top3_acc / total_count)  # (batch_size * (count_2 + 1)))
            running_top5_acc.append(ave_top5_acc / total_count)  # (batch_size * (count_2 + 1))) 
            running_top7_acc.append(ave_top7_acc / total_count) 
            running_top9_acc.append(ave_top9_acc / total_count) 
            running_top11_acc.append(ave_top11_acc / total_count) 
            running_top13_acc.append(ave_top13_acc / total_count)
            running_top15_acc.append(ave_top15_acc / total_count)
            
            print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
            print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
            print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
            print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
            print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
            print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
            print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
            print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
            print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))
            
            cur_accuracy  = running_top1_acc[-1]
   
            print("current acc", cur_accuracy)
            print("best acc", best_accuracy)
            if cur_accuracy > best_accuracy:
                print("Saving the best model")
                net_name = checkpoint_directory  + '//' +  'cnn_beam_pred-64beams'
                t.save(net.state_dict(), net_name)  
                best_accuracy =  cur_accuracy  
            print("updated best acc", best_accuracy)
                
                        
            print("Saving the predicted value in a csv file")
            file_to_save = f'{save_directory}//topk_pred_beam_val_after_{epoch+1}th_epoch.csv'
            indx = np.arange(1, len(top1_pred_out)+1, 1)
            df1 = pd.DataFrame()
            df1['index'] = indx                
            df1['link_status'] = gt_beam
            df1['top1_pred'] = top1_pred_out
            df1['top3_pred'] = top3_pred_out
            df1['top5_pred'] = top5_pred_out
            df1['top7_pred'] = top7_pred_out
            df1['top9_pred'] = top9_pred_out
            df1['top11_pred'] = top11_pred_out
            df1['top13_pred'] = top13_pred_out
            df1['top15_pred'] = top15_pred_out
            df1.to_csv(file_to_save, index=False)
                                      
            LR_sch.step()
            
        top_1[0,idx] = running_top1_acc[-1]
        top_3[0,idx] = running_top3_acc[-1]
        top_5[0,idx] = running_top5_acc[-1]
        top_7[0,idx] = running_top7_acc[-1]
        top_9[0,idx] = running_top9_acc[-1]
        top_11[0,idx] = running_top11_acc[-1]
        top_13[0,idx] = running_top13_acc[-1]
        top_15[0,idx] = running_top15_acc[-1]

In [ ]:
# Testing

In [ ]:
# Load the model checkpoint
test_dir = './scenario36_64_pos_beam_test.csv'

# Load the test data
test_data = pd.read_csv(test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr1_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr1'].tolist()

# Load the model checkpoint and prepare for evaluation
checkpoint_path = f'{checkpoint_directory}/cnn_beam_pred-64beams'
model.load_state_dict(torch.load(checkpoint_path))
model.eval() 
net = model.cuda()

In [ ]:
test_loader = DataLoader(DataFeed(test_dir, transform=proc_pipe),
                            batch_size=val_batch_size,
                            #num_workers=8,
                            shuffle=False)

In [ ]:
print('Start testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []  
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(test_loader):
    net.eval()
    data = pos_data.type(torch.Tensor)  
    x = data.cuda()                    
    labels = beam_val[:,0].type(torch.LongTensor)   
    opt.zero_grad()
    labels = labels.cuda()
    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x.reshape([-1, 2, 1]))
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())
    
    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())
    
    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())
    
    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())
    
    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())
    
    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())
    
    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())
            
    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)
       
    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

In [ ]:
print("total test examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)  # (batch_size * (count_2 + 1)))
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)   
running_top9_acc.append(ave_top9_acc / total_count)   
running_top11_acc.append(ave_top11_acc / total_count)   
running_top13_acc.append(ave_top13_acc / total_count)   
running_top15_acc.append(ave_top15_acc / total_count)   

print('Testing_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval.csv'

indx = test_data.index + 1
df2 = pd.DataFrame()
df2['index'] = org
df2['link_status'] = link_status_data  # Add the link_status column
df2['original_unit1_pwr1'] = pwr_60ghz # Add the original_unit1_pwr_60ghz column
df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)